In [ ]:
# @title Cell 1: Restore workspace from backup of Notebook 1 to Export (<1 Minute)
from google.colab import drive
import os
import shutil
import glob
import zipfile
import random

# Function to restore workspace from the latest backup
def restore_workspace_backup():
    # Ensure Google Drive is mounted
    print("Checking Google Drive mount status...")
    if not os.path.exists('/content/drive'):
        print("Google Drive not mounted. Attempting to mount...")
        try:
            drive.mount('/content/drive')
            print("Google Drive mounted successfully.")
        except Exception as e:
            print(f"Error mounting Google Drive: {e}")
            print("Cannot proceed with restore without Google Drive mounted.")
            return # Exit function if mount fails
    else:
        print("Google Drive is already mounted.")

    drive_backup_dir = "/content/drive/MyDrive/mower_backups"
    print(f"Attempting to restore workspace from zip backup in {drive_backup_dir}...")

    if not os.path.exists(drive_backup_dir):
        print(f"Backup directory not found at {drive_backup_dir}. Skipping restore.")
        print("If this is your first run or you haven't saved a backup, this is expected.")
        return

    # Find the latest zip file based on modification time
    list_of_files = glob.glob(f"{drive_backup_dir}/mower_backup_*.zip")
    if not list_of_files:
        print(f"No workspace backup zip files found in {drive_backup_dir}. Skipping restore.")
        return

    latest_zip_path = max(list_of_files, key=os.path.getmtime)
    print(f"Found latest workspace backup zip: {latest_zip_path}")

    temp_zip_path = "/tmp/latest_workspace_backup.zip"

    try:
        # Copy the zip file to a temporary location in Colab
        print(f"Copying zip file to temporary location: {temp_zip_path}...")
        shutil.copy2(latest_zip_path, temp_zip_path)
        print("Zip file copied successfully.")

        # Ensure the target directory exists for unzipping (root /content)
        os.makedirs('/content', exist_ok=True)
        print("Extracting zip archive to /content...")

        # Extract the zip file
        # WARNING: This will overwrite existing files in /content that are in the zip!
        with zipfile.ZipFile(temp_zip_path, 'r') as zipf:
            # Get list of files in the zip to handle potential exclusions during restore if needed
            # For a full workspace restore, we typically just extract everything.
            zipf.extractall('/content')

        print("Workspace successfully restored.")

    except Exception as e:
        print(f"Error restoring workspace from Google Drive: {e}")

    finally:
        # Clean up the temporary zip file
        if os.path.exists(temp_zip_path):
            os.remove(temp_zip_path)
            print(f"Removed temporary zip file: {temp_zip_path}")

# Call the function to attempt restore
restore_workspace_backup()

# --- After restoring, you might need to re-verify essential paths/variables ---
# This is important because the restoration replaces the /content filesystem state.

# Re-define BASE_DIR and MASTER_CLASS_LIST from Cell 2, as they might be needed
# immediately by subsequent cells, and the restored environment might not have them defined yet
# if this cell is run after a crash and before running Cell 2 again.
# It's safest to re-declare them here if they are critical global variables.
# If they were defined in Cell 2 which you intend to re-run anyway, this might be redundant.
# Assuming BASE_DIR and MASTER_CLASS_LIST were critical for subsequent steps:
if 'BASE_DIR' not in globals() or BASE_DIR is None:
     BASE_DIR = "/content/mower_dataset"
     print(f"Re-initialized BASE_DIR to {BASE_DIR} after restore.")

if 'MASTER_CLASS_LIST' not in globals() or MASTER_CLASS_LIST is None:
    MASTER_CLASS_LIST = ["grass", "dirt", "sand", "mulch", "pavement", "concrete", "gravel", "tree", "shrub", "flower", "planter", "stump", "rock", "hill", "water_feature", "ditch", "pool", "lake", "river", "fountain", "waterfall", "field", "curb", "edging", "fence", "gate", "retaining_wall", "railing", "bench", "bridge", "stairs", "path", "sign", "pole", "lamp_post", "streetlight", "traffic_light", "person", "animal", "dog", "cat", "bicycle", "toy", "tool", "hose", "sprinkler", "swing_set", "slide", "sandbox", "trampoline", "furniture", "decoration", "vehicle", "car", "bus", "truck", "mailbox", "trash_bin", "recycling_bin"]
    master_index = {name: idx for idx, name in enumerate(MASTER_CLASS_LIST)} # Re-create index
    print("Re-initialized MASTER_CLASS_LIST and master_index after restore.")

# You might also need to re-load the data.yaml file path if it was critical
# This is already handled by the `recover_from_drive` function in Cell 10b,
# but if you skipped 10b, you might need something like:
# DATA_YAML_PATH = f"{BASE_DIR}/data.yaml"
# print(f"Set DATA_YAML_PATH to {DATA_YAML_PATH}")

# You might also need to remount Drive explicitly if restore didn't handle it,
# though the restore function attempts to do so.
# drive.mount('/content/drive', force_remount=True) # Use force_remount if needed

# After running this cell, you should then proceed to the cell where the export was attempted
# (likely the cell after 11a which triggered the need for 11b).

In [ ]:
# @title Cell 1b: Create labels.txt

print("Creating labels.txt file...")

# This list should be defined in the previous "Restore" cell
if 'MASTER_CLASS_LIST' in globals():
    with open("labels.txt", "w") as f:
        for class_name in MASTER_CLASS_LIST:
            f.write(class_name + "\n")
    print("✅ labels.txt created successfully at /content/labels.txt")
else:
    print("❌ ERROR: MASTER_CLASS_LIST is not defined. Cannot create labels.txt.")

In [ ]:
# @title Cell 2: Final Setup for Model Export (~3 Minutes)

# 1. Install GPU-enabled PyTorch
print("Installing GPU-enabled PyTorch...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --quiet

# 2. Install TensorFlow, Ultralytics, AND all specific export dependencies
# This adds 'ai-edge-litert' to preempt the final AutoUpdate trigger.
print("\nInstalling TF, Ultralytics, and ALL export requirements...")
!pip install tensorflow ultralytics 'sng4onnx>=1.0.1' 'onnx_graphsurgeon>=0.3.26' 'onnx>=1.12.0,<1.18.0' 'onnx2tf>=1.26.3' 'onnxslim>=0.1.59' onnx-simplifier onnxruntime-gpu 'ai-edge-litert>=1.2.0,<1.4.0' --quiet

# 3. Verify the environment
print("\nVerifying the environment...")
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available? {torch.cuda.is_available()}")

# 4. Install the Edge TPU Compiler
print("\nInstalling Edge TPU Compiler...")
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler -y
print("✅ Edge TPU Compiler installed.")

# Run the official Ultralytics check
print("\nRunning Ultralytics checks:")
import ultralytics
ultralytics.checks()

In [ ]:
# @title Cell 3a: (Optional) Force GPU Usage with Environment Variable
# Run this BEFORE the export cells if GPU still isn't being used

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Also set TensorFlow to use GPU memory growth
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"TensorFlow GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)

In [ ]:
# @title Cell 3: Export Models and Compile for Coral
import os
import shutil
import glob
import random
from ultralytics import YOLO

# --- 1. Create Representative Dataset ---
print("--- Creating Representative Dataset ---")
rep_data_dir = "/content/valid_images/"
os.makedirs(rep_data_dir, exist_ok=True)
BASE_DIR = "/content/mower_dataset"
val_imgs = glob.glob(os.path.join(f"{BASE_DIR}/images/val", "*.jpg"))
if val_imgs:
    # Use a fixed random seed for reproducibility
    random.seed(42)
    for img_path in random.sample(val_imgs, min(150, len(val_imgs))):
        shutil.copy(img_path, rep_data_dir)
    print(f"Created representative dataset with {len(os.listdir(rep_data_dir))} images.")
else:
    print("WARNING: Validation image folder is empty.")

# --- 2. Export Pi Model (Float32 TFLite) ---
print("\n--- Exporting Pi Model ---")
pi_model_pt_path = "/content/mower_model/pi_model_yolov8n10/weights/best.pt"
model_pi = YOLO(pi_model_pt_path)
# Explicitly set output name and format for clarity
model_pi.export(format="tflite", imgsz=640, int8=False, name="pi_model_float32", device=0)
print("✅ Pi model export complete.")

# --- 3. Export Coral Model to Simplified ONNX ---
print("\n--- Exporting Coral Model to Simplified ONNX ---")
coral_model_pt_path = "/content/mower_model/coral_model_yolov8n/weights/best.pt"
coral_onnx_output_path = "/content/mower_model/coral_model_yolov8n/weights/best.onnx"

model_coral = YOLO(coral_model_pt_path)
# Export with nms=False to simplify the graph for the Edge TPU compiler
model_coral.export(format="onnx", imgsz=640, nms=False, name="coral_model_simplified", device=0)
print(f"✅ Coral model simplified ONNX export complete: {coral_onnx_output_path}") # Note: Ultralytics saves ONNX to working dir by default with 'name'

# --- Diagnostic: Check onnx2tf help ---
# print("\n--- Diagnostic: onnx2tf help ---")
# !onnx2tf --help
# print("--- End of Diagnostic ---")
# --- End of Diagnostic ---


# --- 4. Convert Simplified ONNX to INT8 Quantized TFLite ---
print("\n--- Converting Simplified ONNX to INT8 TFLite ---")
quantized_tflite_path = "/content/coral_model_quantized_int8.tflite"
# Explicitly define input and output paths in the command
# Using --quantization=int8 and --dataset-for-calibration based on onnx2tf --help
!onnx2tf -i {coral_onnx_output_path} -o {quantized_tflite_path} -oiqt

# # --- Diagnostic: Check file system after onnx2tf ---
# print("\n--- Diagnostic: Checking file system after onnx2tf ---")
# print(f"Current working directory: {os.getcwd()}")
# print(f"Listing files in /content/ after onnx2tf:")
# !ls /content/
# print("\nRecursively searching for .tflite files in /content/ (excluding /content/drive):")
# !find /content/ -name "*.tflite" -not -path "/content/drive/*"
# print("--- End of Diagnostic ---")
# # --- End of Diagnostic ---


print("✅ ONNX to INT8 TFLite conversion command executed.")

# --- 5. Compile Quantized TFLite for Edge TPU ---
print(f"--- Compiling for Edge TPU ---")
output_dir = "/content/compiled_models/"
os.makedirs(output_dir, exist_ok=True)

# The onnx2tf tool creates a directory. We need to find the correct file inside it.
quantized_tflite_dir = "/content/coral_model_quantized_int8.tflite"
quantized_tflite_src_file = "" # Initialize variable

# Search for the full integer quantized TFLite file
search_pattern = os.path.join(quantized_tflite_dir, "*_full_integer_quant.tflite")
found_files = glob.glob(search_pattern)

if found_files:
    quantized_tflite_src_file = found_files[0] # Get the first match
    print(f"Found quantized model for compilation: {quantized_tflite_src_file}")

    # Check if the input file exists and is not empty
    if os.path.exists(quantized_tflite_src_file) and os.path.getsize(quantized_tflite_src_file) > 0:
        print(f"Input file for Edge TPU compilation is valid (Size: {os.path.getsize(quantized_tflite_src_file)} bytes)")
        !edgetpu_compiler {quantized_tflite_src_file} --out_dir={output_dir}
        print("✅ Edge TPU compilation command executed.")
    else:
        print(f"❌ Error: Input file for Edge TPU compilation is not valid: {quantized_tflite_src_file}")
else:
    print(f"❌ Error: No '*_full_integer_quant.tflite' file found in {quantized_tflite_dir}.")
    print("This is likely because the onnx2tf conversion failed.")

In [ ]:
# @title Cell 4: PUSH TO GITHUB
from google.colab import userdata
import os
import shutil
import glob

# --- PUSH TO GITHUB ---
print("--- PUSHING MODELS TO GITHUB ---")
GITHUB_REPO_URL = "https://github.com/acredsfan/autonomous_mower.git"
GITHUB_BRANCH = "main"
GITHUB_PAT = userdata.get('GITHUB_PAT')

if not GITHUB_PAT:
    print("Error: GitHub Personal Access Token not found in secrets.")
    print("Skipping GitHub push.")
else:
    # Configure Git
    !git config --global user.email "you@example.com"
    !git config --global user.name "Autonomous Mower Trainer"

    push_dir = "push_dir"
    if os.path.exists(push_dir) and os.path.isdir(os.path.join(push_dir, '.git')):
        print(f"Repository directory '{push_dir}' already exists. Attempting to pull latest changes...")
        %cd {push_dir}
        !git pull origin {GITHUB_BRANCH}
        %cd ..
    else:
        if os.path.exists(push_dir):
             print(f"Removing non-git directory '{push_dir}' before cloning.")
             shutil.rmtree(push_dir)
        print(f"Cloning repository {GITHUB_REPO_URL} branch {GITHUB_BRANCH}...")
        !git clone --depth 1 -b {GITHUB_BRANCH} https://{GITHUB_PAT}@github.com/acredsfan/autonomous_mower.git {push_dir}

    if os.path.exists(push_dir) and os.path.isdir(os.path.join(push_dir, '.git')):
        # Define source paths based on the actual output of Cell 3
        pi_tflite_src = "/content/mower_model/pi_model_yolov8n10/weights/best_saved_model/best_float32.tflite"
        coral_onnx_src = "/content/mower_model/coral_model_yolov8n/weights/best.onnx"
        coral_int8_tflite_src = "/content/coral_model_quantized_int8.tflite/best_full_integer_quant.tflite"
        coral_edgetpu_src = "/content/compiled_models/best_full_integer_quant_edgetpu.tflite"

        labels_src = "/content/labels.txt"

        models_dest_dir = os.path.join(push_dir, "models")
        os.makedirs(models_dest_dir, exist_ok=True)
        print(f"Copying generated files to {models_dest_dir}...")

        files_to_copy = {
            pi_tflite_src: "pi_model_float32.tflite",
            coral_onnx_src: "coral_model_simplified.onnx",
            coral_int8_tflite_src: "coral_model_quantized_int8.tflite",
            coral_edgetpu_src: "coral_model_quantized_int8_edgetpu.tflite",
            labels_src: "labels.txt"
        }

        for src, dest_name in files_to_copy.items():
            if src and os.path.exists(src):
                dest_path = os.path.join(models_dest_dir, dest_name)
                shutil.copy2(src, dest_path)
                print(f"Copied {os.path.basename(src)} to {dest_path}")
            else:
                print(f"WARNING: Source file not found for '{dest_name}'. Path: '{src}'. Skipping.")

        # Commit and push changes
        %cd {push_dir}
        print("Adding files to Git...")
        !git add models/*
        # Force add labels.txt if it's being ignored
        if os.path.exists("models/labels.txt"):
            !git add -f models/labels.txt

        git_status = !git status --porcelain
        if git_status:
            print("Committing changes...")
            !git commit -m "Update trained models and compilation results"
            print("Pushing changes to origin...")
            !git push origin {GITHUB_BRANCH}
        else:
            print("No new changes to commit.")

        # Return to the root content directory
        %cd ..
    else:
        print("Error: Repository directory was not created/found. Skipping GitHub push.")

print("\nGitHub push process complete.")

In [ ]:
# @title Cell 5: SAVE MODELS TO GOOGLE DRIVE
import os
import shutil
import glob

# Define the destination folder in your Google Drive
DRIVE_SAVE_DIR = "/content/drive/MyDrive/Mower_Trained_Models"
os.makedirs(DRIVE_SAVE_DIR, exist_ok=True)

print(f"Saving models to Google Drive folder: {DRIVE_SAVE_DIR}")

# Define the source paths for the models generated in Cell 3
# These paths are based on the output of the export and compile steps in Cell 3.
pi_tflite_src = "/content/mower_model/pi_model_yolov8n10/weights/best_saved_model/best_float32.tflite"
coral_onnx_src = "/content/mower_model/coral_model_yolov8n/weights/best.onnx"
coral_int8_tflite_src = "/content/coral_model_quantized_int8.tflite/best_full_integer_quant.tflite"
coral_edgetpu_src = "/content/compiled_models/best_full_integer_quant_edgetpu.tflite"

labels_src = "/content/labels.txt"

models_dest_dir = os.path.join(push_dir, "models")
os.makedirs(models_dest_dir, exist_ok=True)
print(f"Copying generated files to {models_dest_dir}...")

files_to_copy = {
    pi_tflite_src: "pi_model_float32.tflite",
    coral_onnx_src: "coral_model_simplified.onnx",
    coral_int8_tflite_src: "coral_model_quantized_int8.tflite",
    coral_edgetpu_src: "coral_model_quantized_int8_edgetpu.tflite",
    labels_src: "labels.txt"
}

saved_count = 0
for src, dest_name in files_to_copy.items():
    if src and os.path.exists(src):
        dst_path = os.path.join(DRIVE_SAVE_DIR, dest_name)
        try:
            shutil.copy2(src, dst_path)
            print(f"✅ Saved {dest_name} to Google Drive")
            saved_count += 1
        except Exception as e:
            print(f"❌ Error saving {dest_name} to Google Drive: {e}")
    else:
        print(f"❌ Error: Source file not found for Google Drive save: {src}. Skipping.")

print(f"\nModel saving to Google Drive complete. {saved_count} file(s) saved.")
